# Autoregressive covariance structure 

We try to parameterize covariance Gamma only using two parameters rho and sigma2 as in the AR(1) structure.

For n = 50 this autoregressive model converges in 26 iterations using Quasi-Newton, 28 with Newton.

1. First we need to modify the structure itself
2. Second modify the loglikelihood function 
3. Third modify gradient and hessian with respect to beta
4. ADD gradient and hessian with respect to AR(1) parameterization
5. Quasi-Newton works
6. Newton works with cross terms for rho and sigma2 and cross term with beta and sigma2
7. May need cross terms with beta and rho

To Do: Strangely Newton's takes more iterations than Quasi-Newton... Look into the Hessians?

In [1]:
using DataFrames, Random, GLM, GLMCopula, Test
using LinearAlgebra, BenchmarkTools, Revise

Random.seed!(1234)

# sample size
N = 10000
# observations per subject
n = 50
ρ = 0.1
σ2 = 0.1

V = zeros(n, n) # will store the AR(1) structure without sigma2

mean = 5

dist = Poisson

V = get_AR_cov(n, ρ, σ2, V)

# true Gamma
Γ = σ2 * V

50×50 Array{Float64,2}:
 0.1      0.01     0.001    0.0001   …  1.0e-47  1.0e-48  1.0e-49  1.0e-50
 0.01     0.1      0.01     0.001       1.0e-46  1.0e-47  1.0e-48  1.0e-49
 0.001    0.01     0.1      0.01        1.0e-45  1.0e-46  1.0e-47  1.0e-48
 0.0001   0.001    0.01     0.1         1.0e-44  1.0e-45  1.0e-46  1.0e-47
 1.0e-5   0.0001   0.001    0.01        1.0e-43  1.0e-44  1.0e-45  1.0e-46
 1.0e-6   1.0e-5   0.0001   0.001    …  1.0e-42  1.0e-43  1.0e-44  1.0e-45
 1.0e-7   1.0e-6   1.0e-5   0.0001      1.0e-41  1.0e-42  1.0e-43  1.0e-44
 1.0e-8   1.0e-7   1.0e-6   1.0e-5      1.0e-40  1.0e-41  1.0e-42  1.0e-43
 1.0e-9   1.0e-8   1.0e-7   1.0e-6      1.0e-39  1.0e-40  1.0e-41  1.0e-42
 1.0e-10  1.0e-9   1.0e-8   1.0e-7      1.0e-38  1.0e-39  1.0e-40  1.0e-41
 1.0e-11  1.0e-10  1.0e-9   1.0e-8   …  1.0e-37  1.0e-38  1.0e-39  1.0e-40
 1.0e-12  1.0e-11  1.0e-10  1.0e-9      1.0e-36  1.0e-37  1.0e-38  1.0e-39
 1.0e-13  1.0e-12  1.0e-11  1.0e-10     1.0e-35  1.0e-36  1.0e-37  1.0e-38
 

In [2]:
vecd = [dist(mean) for i in 1:n]
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)

Y_Nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, N)

10000-element Array{Array{Float64,1},1}:
 [7.0, 8.0, 7.0, 4.0, 8.0, 2.0, 6.0, 6.0, 6.0, 7.0  …  5.0, 3.0, 4.0, 6.0, 4.0, 5.0, 5.0, 6.0, 4.0, 7.0]
 [3.0, 7.0, 6.0, 3.0, 6.0, 6.0, 3.0, 3.0, 8.0, 5.0  …  3.0, 4.0, 5.0, 8.0, 2.0, 5.0, 4.0, 4.0, 8.0, 6.0]
 [6.0, 3.0, 3.0, 7.0, 4.0, 3.0, 6.0, 2.0, 1.0, 8.0  …  2.0, 4.0, 3.0, 4.0, 6.0, 3.0, 4.0, 4.0, 3.0, 5.0]
 [9.0, 4.0, 5.0, 3.0, 5.0, 5.0, 4.0, 12.0, 4.0, 5.0  …  4.0, 8.0, 7.0, 9.0, 5.0, 3.0, 8.0, 5.0, 4.0, 4.0]
 [5.0, 7.0, 4.0, 7.0, 4.0, 1.0, 7.0, 3.0, 5.0, 4.0  …  9.0, 7.0, 3.0, 1.0, 3.0, 5.0, 3.0, 6.0, 5.0, 5.0]
 [6.0, 6.0, 10.0, 2.0, 4.0, 2.0, 6.0, 7.0, 1.0, 5.0  …  2.0, 6.0, 3.0, 6.0, 3.0, 6.0, 4.0, 4.0, 4.0, 5.0]
 [5.0, 4.0, 6.0, 3.0, 4.0, 6.0, 0.0, 3.0, 3.0, 3.0  …  4.0, 6.0, 3.0, 3.0, 5.0, 1.0, 4.0, 1.0, 5.0, 10.0]
 [6.0, 4.0, 5.0, 7.0, 5.0, 8.0, 5.0, 3.0, 10.0, 1.0  …  5.0, 4.0, 5.0, 3.0, 4.0, 6.0, 5.0, 4.0, 7.0, 2.0]
 [4.0, 5.0, 8.0, 5.0, 5.0, 4.0, 7.0, 3.0, 3.0, 6.0  …  5.0, 2.0, 4.0, 7.0, 6.0, 5.0, 6.0, 5.0, 7.0, 4.0]
 [4.0, 6.0

In [3]:
Random.seed!(1234)

d = Poisson()
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{GLMCopulaARObs{T, D, Link}}(undef, N)

for i in 1:N
    y = Float64.(Y_Nsample[i])
    X = ones(n, 1)
    gcs[i] = GLMCopulaARObs(y, X, d, link)
end

gcm = GLMCopulaARModel(gcs);

In [4]:
initialize_model!(gcm)
@show gcm.β
@show exp.(gcm.β);
fill!(gcm.ρ, 0.0)
fill!(gcm.σ2, 0.0)

loglikelihood!(gcm, true, true)

gcm2 = deepcopy(gcm)
gcm3 = deepcopy(gcm);

initializing β using Newton's Algorithm under Independence Assumption
1 0.0 -1.1093585944799678e6 39999
2 -1.1093585944799678e6 -1.1093585944799678e6 39999
gcm.β = [1.613054963012508]
exp.(gcm.β) = [5.018117999999999]


# Quasi-Newton
## Number of Iterations....: 26
## 27.8 seconds

In [5]:
@time GLMCopula.fit!(gcm, IpoptSolver(print_level = 5, max_iter = 100, warm_start_init_point="yes", hessian_approximation = "limited-memory"))

gcm.θ = [1.613054963012508, 0.0, 0.0]
gcm.∇θ = [2.5979574047596543e-10, 0.0, 7034.695979448923]
gcm.θ = [1.613054963012508, 0.00999999, 0.00999999]
gcm.∇θ = [-2013.5757220422408, 10.383726613141, 3052.1035044366386]

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       

-1.1092594676023386e6

# Newton 
## Number of Iterations....: 47
## 74 seconds

In [6]:
@time GLMCopula.fit!(gcm2, IpoptSolver(print_level = 5, max_iter = 100, warm_start_init_point="yes", hessian_approximation = "exact"))

gcm.θ = [1.613054963012508, 0.0, 0.0]
gcm.∇θ = [2.5979574047596543e-10, 0.0, 7034.695979448923]
gcm.θ = [1.613054963012508, 0.00999999, 0.00999999]
gcm.∇θ = [-2013.5757220422408, 10.383726613141, 3052.1035044366386]
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        5

Total number of variables............................:        3
                     variables with only lower bounds:        1
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0


  31  1.1092595e+06 0.00e+00 1.51e-05  -5.7 1.18e-08    -  1.00e+00 5.00e-01f  2
gcm.θ = [1.6102620422239233, 0.09493264902739856, 0.0934100136056889]
gcm.∇θ = [0.009183439608277943, -1.6901426938553676e-6, -1.8138626739272468e-5]
  32  1.1092595e+06 0.00e+00 1.31e-04  -8.6 2.99e-06    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.610262044124232, 0.09493265056477587, 0.09341000822351067]
gcm.∇θ = [0.004703452944117892, -1.6302878218799055e-6, -4.7100902945018674e-5]
  33  1.1092595e+06 0.00e+00 6.69e-05  -8.6 1.08e-08    -  1.00e+00 5.00e-01f  2
gcm.θ = [1.6102620460846004, 0.09493265712295162, 0.09340998606690801]
gcm.∇θ = [0.0004394475221971561, -1.614841056152283e-6, -4.0147324162909115e-5]
  34  1.1092595e+06 0.00e+00 6.25e-06  -8.6 2.22e-08    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.6102620462805088, 0.0949326623379134, 0.09340996864096303]
gcm.∇θ = [0.00034091080502385296, -1.65742233854127e-6, -5.710682269288725e-6]
  35  1.1092595e+06 0.00e+00 4.85e-06  -8.6 1.74e-08    -  1.00e+00 1.00e+00

-1.1092594676023452e6

# Newton with mehrotra
## Number of Iterations....: 25
## 31 seconds 

In [7]:
@time GLMCopula.fit!(gcm3, IpoptSolver(print_level = 5, max_iter = 100, mehrotra_algorithm="yes", warm_start_init_point="yes", hessian_approximation = "exact"))

gcm.θ = [1.613054963012508, 0.0, 0.0]
gcm.∇θ = [2.5979574047596543e-10, 0.0, 7034.695979448923]
gcm.θ = [1.613054963012508, 0.199999994, 9.99999999]
gcm.∇θ = [-9386.860880070397, -107.66898664947577, -0.07224384799637316]
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        5

Total number of variables............................:        3
                     variables with only lower bounds:        1
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:    

-1.109259467602339e6

In [8]:
@show gcm.∇θ
@show gcm2.∇θ
@show gcm3.∇θ;

gcm.∇θ = [-3.4067943044391313e-7, -1.8501513654456403e-9, -1.4240901791140459e-8]
gcm2.∇θ = [4.173215435443467e-7, -6.029513112681961e-7, -7.195042046603817e-7]
gcm3.∇θ = [-4.5735852616246575e-7, -6.650083261838802e-9, 6.32383567733541e-9]


In [9]:
@show gcm.θ
@show gcm2.θ
@show gcm3.θ;

gcm.θ = [1.6102620464578044, 0.09493265929565019, 0.09340996457130182]
gcm2.θ = [1.6102620464551523, 0.09493266075867558, 0.09340996483569065]
gcm3.θ = [1.6102620464579538, 0.09493265931127859, 0.09340996455994997]


In [10]:
@show loglikelihood!(gcm, true, true)
@show loglikelihood!(gcm2, true, true)
@show loglikelihood!(gcm3, true, true);

loglikelihood!(gcm, true, true) = -1.1092594676023386e6
loglikelihood!(gcm2, true, true) = -1.1092594676023452e6
loglikelihood!(gcm3, true, true) = -1.109259467602339e6
